In [1]:
import importlib
import pandas
import numpy
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
"""
df_raw - база даних завершених закупівель 24 обласних центрів з 1 серпня 2016 до 31 грудня 2019.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Згерерована у Профейсійному модулі аналітики bipro.prozorro.
"ИдентификаторЛота" є унікальним ідентифікатором рядка.
"""  

In [3]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_raw.columns
print(df_raw.shape[0])

537893


### Step 0:
#### Створити базу навчальних закладів, які самостійно купують через ProZorro

In [4]:
step_mark = 'step0'
action_mark = 'check'
number_0 = '0'
columns_to_check_step0 = ['Организатор']

# Перший спосіб відібрати навчальні заклади, які є замовниками: пошук за ЄДРПОУ в полі "IDOrganizator"
df_tenders_by_schools_by_edrpou = filter_dataframe(df_raw, positive_edrpous(), ['IDOrganizator'])

# Створити базу для закупівель навчальних закладів, які НЕ є замовниками
df_merged_left = pandas.merge(df_raw, df_tenders_by_schools_by_edrpou, on='ИдентификаторЛота', 
                              how = 'left', indicator = True, suffixes = ('','_temp'))
df_undefined_0 = df_merged_left[df_merged_left._merge=='left_only']
df_undefined_0 = df_undefined_0[training_data_columns]

# Другий спосіб відібрати навчальні заклади, які є замовниками: пошук за ключовими словами в полі "Организатор"
df_step0 = df_undefined_0.copy()

for column in columns_to_check_step0:
    df_step0 = step_check(df_step0, column, number_0)
    
df_tenders_by_schools_by_keywords_0, df_undefined_0 = separate_positives_and_negatives_ogranizator(df_step0, step_mark, action_mark)
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords:", df_tenders_by_schools_by_keywords_0.shape[0])

/home/artem/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


537893 rows. filtered 38817
step0: df_tenders_by_schools_by_keywords: 12598


#### Concatenating 2 dataframes: df_tenders_by_schools_concatenated

In [5]:
# df_tenders_by_schools_concatenated - 
# Об'єднання двох баз закупівель навчальних закладів, які самостійно купують через ProZorro

frames_tenders_by_schools = [df_tenders_by_schools_by_edrpou, df_tenders_by_schools_by_keywords_0]
df_tenders_by_schools_concatenated = pandas.concat(frames_tenders_by_schools)
len_df = df_tenders_by_schools_concatenated.shape[0]
df_tenders_by_schools_concatenated.drop_duplicates(subset = 'ИдентификаторЛота', inplace = True)
print("Removed duplicates:", len_df - df_tenders_by_schools_concatenated.shape[0])

Removed duplicates: 353


### Step 1: 
#### Створити базу закупівель навчальних закладів, які купують у ProZorro через посередників: управління освіти, міністерства

In [ ]:
# TODO Step 1:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "АДЕРСА ПОСТАВКИ"

"""
За результатами точності визначення закупівель вирішити чи фільтрувати за цією колонкою.
Проблема: в полі адреса часто вказують ключові слова, які ніби можуть вказувати на приналежність
закупівлі до шкільних. Наприклад, вул. Шкільна, Гімназійна і т.п.
"""

In [6]:
step_mark = 'step1'
action_mark = 'check'
number_1 = '1'
columns_to_check_step1 = ['Тендер', 'ОписаниеТендера', 'Лот']
df_step1 = df_undefined_0.copy()

for column in columns_to_check_step1:
    df_step1 = step_check(df_step1, column, number_1)

df_tenders_by_schools_by_keywords_1, df_undefined_1 = separate_positives_and_negatives(df_step1, step_mark, action_mark)
df_tenders_by_schools_by_keywords_1 = df_tenders_by_schools_by_keywords_1[training_data_columns]
df_not_school_tenders = df_undefined_1[training_data_columns]
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords_1:", df_tenders_by_schools_by_keywords_1.shape[0])

step1: df_tenders_by_schools_by_keywords_1: 20218


In [ ]:
# TODO:
#    Inspect df_not_school_tenders.csv
#   df_not_school_tenders.to_csv('df_not_school_tenders.csv');

### Step 2:
#### Класифікація навчальних закладів, які самостійно купують через ProZorro

In [7]:
# TODO:
# Визначити пріоритетність фільтрів для визначення категорії. Наприклад, якщо в назві є НВК і дитячий садок.

def classification(given_df, columns_to_classify, filter_list, filter_name):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.contains('|'.join(filter_list), 
                                                                                               case = False, 
                                                                                               na = False)
    for i in range(given_df.shape[0]):
        for column in columns_to_classify:
            if given_df.loc[given_df.index[i], column + '_' + step_mark + '_classification'].any():
                given_df.loc[given_df.index[i], 'Тип закладу'] = filter_name
                break;
    for column in columns_to_classify:
        given_df = clear_redundant_columns(given_df, step_mark, action_mark, column)
    return given_df

In [8]:
step_mark = 'step2'
action_mark = 'classification'
columns_to_classify_step2 = ['Организатор']
df_step2 = df_tenders_by_schools_concatenated.copy()

In [9]:
df_step2_random = df_step2.copy().sample(1000)

#### Filtering

In [10]:
for idx in range(len(filter_priority)):
    df_step2_random = classification(df_step2_random, columns_to_classify_step2, filter_priority[idx], filter_name_priority[idx])

#### Checking results

In [11]:
df_step2_random[df_step2_random['Тип закладу'].notnull()].shape

(615, 13)

In [13]:
print(df_step2_random[df_step2_random['Тип закладу'].isnull()].shape)

columns_to_classify_step2_temp = columns_to_classify_step2.copy()
columns_to_classify_step2_temp.append('Тип закладу')

df_step2_random[df_step2_random['Тип закладу'].isnull()][columns_to_classify_step2_temp]

(385, 13)


,Организатор,Тип закладу
530991,Відділ освіти Сихівського та Личаківського рай...,NaN
40123,Відділ освіти Шевченківського та Залізничного ...,NaN
28332,Управління освіти Подільської районної в м.Киє...,NaN
528438,Управління освіти Голосіївської районної в міс...,NaN
310621,Школа І-ІІІ ступенів № 306 Деснянського району...,NaN
...,...,...
46951,Управління освіти Дарницької районної в місті ...,NaN
380315,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,NaN
99321,Дитячо-юнацька спортивна школа з академічного ...,NaN
404023,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,NaN


In [14]:
df_step2_random[df_step2_random['Тип закладу'] == 'ДНЗ'][columns_to_classify_step2_temp].sample(10)

,Организатор,Тип закладу
224367,Житомирський спеціальний центр розвитку дитини...,ДНЗ
401625,Житомирський дошкільний навчальний заклад №49,ДНЗ
515633,Житомирський дошкільний навчальний заклад № 15,ДНЗ
266030,Херсонський ясла- садок № 38 комбінованого тип...,ДНЗ
460574,"КОМУНАЛЬНИЙ ЗАКЛАД ""ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛ...",ДНЗ
289684,Заклад дошкільної освіти ясла-садок №37 Рівнен...,ДНЗ
288345,Сумський дошкільний навчальний заклад (центр р...,ДНЗ
220245,"КП ""НВО ЛШДНЗ ""Вікторія-П""""",ДНЗ
222223,"Комунальний заклад ""Дошкільний навчальний закл...",ДНЗ
134640,Житомирський дошкільний навчальний заклад № 35,ДНЗ


### Step 3:
#### Класифікація навчальних закладів, для яких купують управління освіти, міністерства чи інші органи влади

In [9]:
# TODO:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "ОРГАНИЗАТОР"

In [15]:
step_mark = 'step3'
action_mark = 'classification'
columns_to_classify_step3 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step3 = df_tenders_by_schools_by_keywords_1.copy()

In [16]:
df_step3_random = df_step3.copy().sample(1000)

#### Filtering

In [17]:
for idx in range(len(filter_priority)):
    df_step3_random = classification(df_step3_random, columns_to_classify_step3, 
                                    filter_priority[idx], filter_name_priority[idx])

#### Checking results

In [18]:
columns_to_classify_step3_temp = columns_to_classify_step3.copy()
columns_to_classify_step3_temp.append('Тип закладу')

In [19]:
df_step3_random['Тип закладу'].value_counts()

ДНЗ                                         400
Загальноосвітня школа                       133
Гімназія                                     60
Навчально-виховний комплекс (об'єднання)     49
Спеціалізована школа                         32
Інтернат                                      6
Ліцей                                         4
Загальносвітня санаторна школа                1
Спеціальна загальноосвітня школа              1
Name: Тип закладу, dtype: int64

In [20]:
df_step3_random[df_step3_random['Тип закладу'].notnull()].shape

(686, 12)

In [21]:
print(df_step3_random[df_step3_random['Тип закладу'].isnull()].shape)
df_step3_random[df_step3_random['Тип закладу'].isnull()][columns_to_classify_step3_temp]

(314, 12)


,Тендер,ОписаниеТендера,Лот,Адрес поставки,Тип закладу
307154,"«Послуги з ремонту, технічного обслуговування ...",NaN,"«Послуги з ремонту, технічного обслуговування ...","вул. Бикова, буд. 7",NaN
153908,Шкільні підручники,NaN,Шкільні підручники,Танкопія 15/2,NaN
473865,«Шкільні меблі» код по ДК 021:2015 – 39160000-...,Джерело фінансування - освітня субвенція,«Шкільні меблі» код по ДК 021:2015 – 39160000-...,"вул. Б. Хмельницького, буд. 28",NaN
525738,Поточно-аварійний ремонт підлоги ЗНЗ № 35,NaN,Поточно-аварійний ремонт підлоги ЗНЗ № 35,"Холодноярський майдан, 1",NaN
20220,технічний нагляд за реалізацією РП «Капіталь...,NaN,технічний нагляд за реалізацією РП «Капіталь...,"вулиця І .Франка, 16",NaN
...,...,...,...,...,...
76587,"Розробка робочого проекту на ""Капітальний ремо...","Розробка робочого проекту на ""Капітальний ремо...","Розробка робочого проекту на ""Капітальний ремо...","вул. Грибоєдова, 22",NaN
489164,30230000-0 Комп’ютерне обладнання,30230000-0 Комп’ютерне обладнання,Лот 1 - Комп’ютерне обладнання (Багатофункціон...,"вул. М. Омеляновича-Павленка, 15",NaN
419845,Поточний ремонт приміщень в СЗШ № 103 Дніпровс...,NaN,Поточний ремонт приміщень в СЗШ № 103 Дніпровс...,"вул. Алма-Атинська, 89",NaN
130499,Технічний нагляд (Капітальний ремонт неслужбов...,NaN,Технічний нагляд (Капітальний ремонт неслужбов...,"вул. Бучми, 18-Д",NaN


### Step 4:
####  присвоєння навчальним закладам власних назв

In [216]:
step_mark = 'step4'
action_mark = 'naming'

columns_to_classify_step4_tenders_by_schools = ['Организатор']
df_step4_tenders_by_schools = df_tenders_by_schools_concatenated.copy()

columns_to_classify_step4 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step4 = df_step3.copy()

In [217]:
def naming(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                                              flags = re.IGNORECASE).str[0].str[-1]
    return given_df

In [218]:
df_step4_tenders_by_schools_filtered = naming(df_step4_tenders_by_schools, columns_to_classify_step4_tenders_by_schools, reg_quotes)
df_step4_tenders_by_schools_filtered.sample(10)[['Организатор', 'Организатор_step4_naming']]

,Организатор,Организатор_step4_naming
330215,НВК № 39,NaN
168949,Житомирський дошкільний навчальний заклад № 56,NaN
363656,Відділ освіти Сихівського та Личаківського рай...,NaN
530299,Дитячо-юнацька спортивна школа №24 Шевченківсь...,NaN
93870,"Загальноосвітній навчальний заклад ""Київський ...",Київський спортивний ліцей-інтернат
367420,Житомирський дошкільний навчальний заклад № 46,NaN
28915,"Хмельницьке навчально-виховне об'єднання №1 ""Ш...",Школа - дитячий садок
439850,Управління освіти Дарницької районної в місті ...,NaN
94752,Житомирський центр розвитку дитини № 69,NaN
270564,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,NaN


In [219]:
df_step4_filtered = naming(df_step4, columns_to_classify_step4, reg_quotes)
df_step4_filtered.sample(10)

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Тендер_step4_naming,ОписаниеТендера_step4_naming,Лот_step4_naming,Адрес поставки_step4_naming
82896,UA-2017-03-16-000573-b-L25,UA-2017-03-16-000573-b,Управління освіти Херсонської міської ради,Овочі та фрукти,5059.97,NaN,"Лот №25 овочі та фрукти, НВК №8",за адресами закладів управління освіти Херсонс...,"вул. Воронцовська, 15-А",Херсонська міська рада,2146713,NaN,NaN,NaN,NaN
16517,UA-2017-12-18-000334-a-L1,UA-2017-12-18-000334-a,Управління капітального будівництва Чернігівсь...,"Реконструкція об'єкту згідно проекту: ""Технічн...",265938.97,"Реконструкція об'єкту згідно проекту: ""Технічн...","Реконструкція об'єкту згідно проекту: ""Технічн...","вул. Івана Мазепи, 19","вул. Івана Мазепи, 19, к. 216",Чернігівська міська рада,5517729,Технічне переоснащення освітлення ДНЗ № 4 по в...,Технічне переоснащення освітлення ДНЗ № 4 по в...,Технічне переоснащення освітлення ДНЗ № 4 по в...,NaN
505689,UA-2019-04-24-001992-b-L1,UA-2019-04-24-001992-b,Управління освіти адміністрації Московського р...,Технічний нагляд за капітальним ремонтом части...,2176.46,NaN,Технічний нагляд за капітальним ремонтом части...,"проспект Тракторобудівників, 110","вул. Валентинівська, 27-г",Харківська міська рада,2146334,NaN,NaN,NaN,NaN
428756,UA-2019-10-25-002429-b-L1,UA-2019-10-25-002429-b,Відділ комунального господарства Краматорської...,Поточний ремонт пішохідного тротуару по вул. С...,175381.2,NaN,Поточний ремонт пішохідного тротуару по вул. С...,Поточний ремонт пішохідного тротуару по вул. С...,"вул. МАШИНОБУДІВНИКІВ, будинок 23",Краматорська міська рада,40477481,NaN,NaN,NaN,NaN
500068,UA-2019-03-15-000070-c-L1,UA-2019-03-15-000070-c,ВК Управління освіти виконавчого комітету Полт...,Виготовлення проектної документації по Капітал...,9435.79,NaN,Виготовлення проектної документації по Капітал...,"вул. Соборності, 36","вул. Соборності, 36",Полтавська міська рада,2145725,NaN,NaN,NaN,NaN
264756,UA-2018-11-21-001617-c-L57,UA-2018-11-21-001617-c,Управління освіти Херсонської міської ради,"Яловичина, свинина, печінка яловичини, тушка к...",118020,NaN,"Лот №57 яловичина, свинина, печінка яловичини,...",за адресами закладів управління освіти Херсонс...,"вул. Воронцовська, 15-А",Херсонська міська рада,2146713,NaN,NaN,NaN,NaN
9585,UA-2017-11-10-000320-b-L1,UA-2017-11-10-000320-b,Управління освіти Херсонської міської ради,Поточний ремонт віконних пройомів спортивної з...,14995.32,NaN,Поточний ремонт віконних пройомів спортивної з...,"вул. Стрітенська, 17","вул. Воронцовська, 15-А",Херсонська міська рада,2146713,Загальноосвітня школа ІІ ступеня - Ліцей журна...,NaN,Загальноосвітня школа ІІ ступеня - Ліцей журна...,NaN
456707,UA-2019-08-19-000591-c-L1,UA-2019-08-19-000591-c,ВК Управління освіти виконавчого комітету Полт...,Поточний ремонт мережі центрального опалення Г...,8505.36,NaN,Поточний ремонт мережі центрального опалення Г...,"вул. Соборності, 36","вул. Соборності, 36",Полтавська міська рада,2145725,NaN,NaN,NaN,NaN
161079,UA-2018-08-22-001420-a-L1,UA-2018-08-22-001420-a,Територіальний відділ освіти Олександрівського...,Інші завершальні будівельні роботи (Поточний р...,50000,NaN,Інші завершальні будівельні роботи (Поточний р...,"вул.Покровська, 29","вул. Поштова, 64",Запорізька міська рада,37611422,NaN,NaN,NaN,NaN
14615,UA-2017-11-30-002945-c-L1,UA-2017-11-30-002945-c,Управління освіти адміністрації Новобаварськог...,Роботи по здійсненню капітального ремонту туал...,196620.85,Роботи по здійсненню капітального ремонту туал...,Роботи по здійсненню капітального ремонту туал...,"вул. Академіка Богомольця,15-А","вул.Ярославська,буд.21",Харківська міська рада,2146311,NaN,NaN,NaN,NaN


### Step 5:
####  присвоєння номерів

In [220]:
step_mark = 'step5'
action_mark = 'numeration'

columns_to_classify_step5_tenders_by_schools = ['Организатор']
df_step4_tenders_by_schools = df_tenders_by_schools_concatenated.copy()

columns_to_classify_step5 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step5 = df_step3.copy()

In [233]:
def numeration(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                                              flags = re.IGNORECASE)
    return given_df

In [234]:
df_step5_tenders_by_schools_filtered = numeration(df_step4_tenders_by_schools, columns_to_classify_step4_tenders_by_schools, reg_number)
df_step5_tenders_by_schools_filtered

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Организатор_step0_check,Организатор_step5_numeration
26,UA-2016-12-28-000118-c-L1,UA-2016-12-28-000118-c,ХМЕЛЬНИЦЬКА СПЕЦІАЛІЗОВАНА ЗАГАЛЬНООСВІТНЯ ШКО...,комп'ютерна техніка,154500,NaN,комп'ютерна техніка,"Кам'янецька,164","ВУЛИЦЯ КАМ'ЯНЕЦЬКА, будинок 164",Хмельницька міська рада,22764070,NaN,[№19]
28,UA-2016-12-29-000048-c-L1,UA-2016-12-29-000048-c,"КОМУНАЛЬНИЙ ЗАКЛАД ""ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬ...",Багатофункціональний пристрій,199000,"БФПCanon IR-2204, БФП Canon i-SENSYS MF217W",Багатофункціональний пристрій,"Дубнівська,32","Дубнівська, 32",Луцька міська рада,21733534,NaN,[]
37,UA-2016-12-29-000053-c-L1,UA-2016-12-29-000053-c,"КОМУНАЛЬНИЙ ЗАКЛАД ""ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬ...",Комп'ютери,199900,Комп'ютер Intel G1840/4G ОЗУ/500G HDD/LG 19M38...,Комп'ютери,"Дубнівська,32","Дубнівська, 32",Луцька міська рада,21733534,NaN,[]
38,UA-2016-12-21-001132-b-L1,UA-2016-12-21-001132-b,Комунальна установа Сумська загальноосвітня ш...,Комп'ютери,128100,NaN,Комп'ютери,"вул.Леваневського,8","вул. Леваневського, 8",Сумська міська рада,21117924,NaN,[№18]
64,UA-2016-12-06-000957-b-L1,UA-2016-12-06-000957-b,Тернопільська загальноосвітня школа І-ІІІ ступ...,Поточний ремонт (заміна вікон),31440,Поточний ремонт (заміна вікон),Поточний ремонт (заміна вікон),"бульвар П. Куліша, 9","бульвар П.Куліша, 9",Тернопільська міська рада,14039951,NaN,[№ 26]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
538482,UA-2019-04-16-000280-c-L1,UA-2019-04-16-000280-c,Комплексна дитячо-юнацька спортивна школа №15,Перевезення команд спортсменів під час провед...,50000,Перевезення команд спортсменів під час провед...,Перевезення команд спортсменів під час провед...,Якубовського 7 а,"Маршала Якубовського,7А",КМДА,22879582,True,[№15]
538507,UA-2019-05-13-001448-c-L1,UA-2019-05-13-001448-c,Комунальний заклад «Спеціалізована школа І-ІІІ...,Перевезення учасників заходу Всеукраїнської ди...,6000,NaN,Перевезення учасників заходу Всеукраїнської ди...,"Миколи закревського, 37-Б","вул. Миколи Закревського, буд. 37-Б",КМДА,3598788,True,[№ 247]
538511,UA-2019-05-14-000325-b-L1,UA-2019-05-14-000325-b,ДИТЯЧА МУЗИЧНА ШКОЛА № 1 М. ХЕРСОНА,Ноутбуки ASUSX540U,29200,Ноутбуки ASUSX540U,Ноутбуки ASUSX540U,NaN,"73000, Херсонська область, місто Херсон, вулиц...",Херсонська міська рада,5533681,True,[№ 1]
538522,UA-2019-04-01-000225-a-L1,UA-2019-04-01-000225-a,Централізована бухгалтерія дитячо-юнацьких спо...,Човни (Човни спортивні байдарки для ДЮСШ «Рега...,398000,NaN,Човни (Човни спортивні байдарки для ДЮСШ «Рега...,м. Київ,"проспект П.Тичини,18",КМДА,26476487,True,[]


In [235]:
df_step5_filtered = numeration(df_step4, columns_to_classify_step4, reg_number)
df_step5_filtered.sample(5)

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Тендер_step4_naming,ОписаниеТендера_step4_naming,Лот_step4_naming,Адрес поставки_step4_naming,Тендер_step5_numeration,ОписаниеТендера_step5_numeration,Лот_step5_numeration,Адрес поставки_step5_numeration
520894,UA-2019-01-25-003831-b-L1,UA-2019-01-25-003831-b,Відділ культури Сєвєродонецької міської ради,Проектування автоматичної установки пожежної с...,6707.92,"за рахунок коштів загального фонду КПНЗ ""БДМШ""",Проектування автоматичної установки пожежної с...,"вул. Озерна, 4","бульвар Дружби Народів, 32а",Сєвєродонецька міська рада,21823131,,БДМШ,,NaN,[],[],[],[]
18382,UA-2017-03-09-000488-a-L1,UA-2017-03-09-000488-a,Державний навчальний заклад «Київське регіонал...,Газети (Періодичні та поліграфічні видання-під...,19541,Запропонований Учасником товар повинен відпові...,Газети (Періодичні та поліграфічні видання-під...,"вул. Маричанська,4","вул. Маричанська, 4",КМДА,37308938,NaN,,NaN,NaN,[],[],[],[]
243641,UA-2018-06-21-000383-b-L1,UA-2018-06-21-000383-b,ВК Управління освіти виконавчого комітету Полт...,Послуги по поточно-аварійному ремонту мережі е...,3521.35,NaN,Послуги по поточно-аварійному ремонту мережі е...,"пров. Токарний, 4","вул. Соборності, 36",Полтавська міська рада,2145725,NaN,NaN,NaN,NaN,[№ 40],NaN,[№ 40],[]
62081,UA-2017-04-04-000193-c-L1,UA-2017-04-04-000193-c,Управління освіти Деснянської районної в місті...,Поліетиленові мішки та пакети для сміття (мішк...,18540,Поставка товару здійснюється в 72 установи Дес...,Поліетиленові мішки та пакети для сміття (мішк...,"вул. Закревського, 15-А","вул. Закревського, 15-А",КМДА,37501684,NaN,NaN,NaN,NaN,[],[],[],[]
207604,UA-2018-03-27-002294-b-L1,UA-2018-03-27-002294-b,ВК Управління освіти виконавчого комітету Полт...,Експертиза кошторисної документації проекту бу...,2109.36,NaN,Експертиза кошторисної документації проекту бу...,"вул. Соборності, 36","вул. Соборності, 36",Полтавська міська рада,2145725,NaN,NaN,NaN,NaN,[№ 26],NaN,[№ 26],[]


### Step 6:
####  зведення в єдину базу

### Optional step

#### За ЄДРПОУ видалити рядки із закупівлями закладів, які не є закладами загальної середньої чи дошкільної освіти

In [ ]:
# TODO: 
# До переліку "негативних" ЄДРПОУ включені управління освіти. 
# Тому заклади, де є "управління освіти" в полі організатор, не видаляти

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])['IDOrganizator'].unique().tolist()

In [ ]:
#filter_dataframe_inspect_before_drop_step_0['Организатор'].unique().tolist()

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, filter_poza, ['Организатор'])['Организатор'].unique().tolist()

In [10]:
#df_step2_positives = filter_dataframe_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])
#df_step2_positives = filter_dataframe_drop(df_step2_positives, filter_poza, ['Организатор'])
#print('Залишилось закупівель навчальних закладів, які не є замовниками:', df_step2_positives.shape[0])